# Create our QandA application

In [ ]:
pip install langchain-community docarray

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from google.colab import drive
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain.llms import HuggingFaceHub

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file = '/content/drive/MyDrive/DSM COURSE/NOTEBOOK/G. LLMs/LangChain/LLM Development/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [ ]:
# Use an open-source embedding model
api_token = "your token"

embeddings = HuggingFaceHubEmbeddings(
    repo_id="sentence-transformers/all-MiniLM-L6-v2",  # Embedding-compatible model
    huggingfacehub_api_token=api_token
)
# Create the index with embeddings
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings  # Specify the embedding model
).from_loaders([loader])

In [ ]:
llm = HuggingFaceHub(
    repo_id='google/flan-t5-small',
    model_kwargs = {'temperature':0.0,'max_length':512},
    huggingfacehub_api_token=api_token
)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

# Coming up with test datapoints

In [ ]:
data[10]

Document(metadata={'source': '/content/drive/MyDrive/DSM COURSE/NOTEBOOK/G. LLMs/LangChain/LLM Development/OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [ ]:
data[11]

Document(metadata={'source': '/content/drive/MyDrive/DSM COURSE/NOTEBOOK/G. LLMs/LangChain/LLM Development/OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

# Hard-coded examples

In [ ]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# LLM-Generated examples

In [ ]:
from langchain.evaluation.qa import QAGenerateChain

In [ ]:
example_gen_chain = QAGenerateChain.from_llm(llm)

In [ ]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

In [ ]:
new_examples[0]

In [ ]:
data[0]

Document(metadata={'source': '/content/drive/MyDrive/DSM COURSE/NOTEBOOK/G. LLMs/LangChain/LLM Development/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

# Combine examples

In [ ]:
examples += new_examples

In [ ]:
qa.run(examples[0]["query"])

<ipython-input-50-b75cb5ccc0a7>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(examples[0]["query"])




> Entering new RetrievalQA chain...

> Finished chain.


'Yes'

# Manual Evaluation

In [ ]:
import langchain
langchain.debug = True

In [ ]:
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 73\nname: Cozy Cuddles Knit Pullover Set\ndescription: Perfect for lounging, this knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out. \n\nSize & Fit \nPants are Favorite Fit: Sits lower on the waist. \nRelaxed Fit: Our most generous fit sits farthest from the body. \n\nFabric & Care \nIn the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features \nRelaxed fit top with raglan sleeves and rounded hem. \nPull-on pants have 

'Yes'

In [ ]:
# Turn off the debug mode
langchain.debug = False

# LLM assisted evaluation

In [ ]:
predictions = qa.apply(examples)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
llm = HuggingFaceHub(
    repo_id='google/flan-t5-small',
    model_kwargs = {'temperature':0.0,'max_length':512},
    huggingfacehub_api_token=api_token
)
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [ ]:
graded_outputs

[{'results': 'INCORRECT'}, {'results': 'INCORRECT'}]

In [ ]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes
Predicted Grade: INCORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: DownTek
Predicted Grade: INCORRECT

